# Preprocesamiento de archivos

## Paso 0

* Bajar los audios
* Descomprimir
* Instalar librería
* Importar librerías

Origen del dataset y datos adicionales: https://zenodo.org/record/1188976#.YMlDGO1Khxd

In [1]:
!wget https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip

--2021-06-22 01:42:20--  https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208468073 (199M) [application/octet-stream]
Saving to: ‘Audio_Speech_Actors_01-24.zip.1’

   Audio_Speech_Act  32%[=====>              ]  64.18M   624KB/s    eta 5m 44s ^C


In [2]:
!sudo apt-get install unzip
!mkdir audios
!unzip Audio_Speech_Actors_01-24.zip -d ./audios/ravdess

!sudo apt-get install sox
!pip install opensmile

/bin/sh: sudo: command not found
mkdir: cannot create directory ‘audios’: File exists
Archive:  Audio_Speech_Actors_01-24.zip
replace ./audios/ravdess/Actor_01/03-01-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
/bin/sh: sudo: command not found
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!wget https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip

--2021-06-22 01:45:13--  https://zenodo.org/record/1188976/files/Audio_Song_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225505317 (215M) [application/octet-stream]
Saving to: ‘Audio_Song_Actors_01-24.zip.1’

          Audio_Son   0%[                    ]       0  --.-KB/s               ^C


In [4]:
!unzip Audio_Song_Actors_01-24.zip -d ./audios/ravdess

Archive:  Audio_Song_Actors_01-24.zip
replace ./audios/ravdess/Actor_01/03-02-01-01-01-01-01.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
from pathlib import Path

import glob
import pandas as pd
import opensmile
import os
import time
import tqdm

## Paso 2
### Recorrer estructuras de archivos y extraer metadata

Filename identifiers 

* Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
* Vocal channel (01 = speech, 02 = song).
* Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
* Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
* Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
* Repetition (01 = 1st repetition, 02 = 2nd repetition).
* Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [2]:
ravdess = []
for file_path in glob.glob('./audios/ravdess/Actor_*/*.wav'):
    filename = Path(file_path).stem
    filename_parts = filename.split('-')
    metadata = {'file_path': file_path,
              'modality': filename_parts[0],
              'vocal_channel': filename_parts[1],
              'emotion': filename_parts[2],
              'emotional_intensity': filename_parts[3],
              'statement': filename_parts[4],
              'repetition': filename_parts[5],
              'actor': filename_parts[6]}
    ravdess.append(metadata)
    pd.DataFrame(ravdess)
    
df_ravdess = pd.DataFrame(ravdess)

In [7]:
df_ravdess.head()

,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor
0,./audios/ravdess/Actor_23/03-01-07-02-02-01-23...,03,01,07,02,02,01,23
1,./audios/ravdess/Actor_23/03-02-03-02-02-01-23...,03,02,03,02,02,01,23
2,./audios/ravdess/Actor_23/03-01-04-01-02-01-23...,03,01,04,01,02,01,23
3,./audios/ravdess/Actor_23/03-02-04-02-02-01-23...,03,02,04,02,02,01,23
4,./audios/ravdess/Actor_23/03-01-02-02-02-02-23...,03,01,02,02,02,02,23


### Extracción de atributos del habla

Extraer el conjunto de atributos eGeMAPS utilizando la librería opensmile-python (**Ref**: https://github.com/audeering/opensmile-python )

#### Atributos de bajo nivel (LLD)


In [8]:
start = time.time()

feat_aux = []
for i in os.listdir('./audios/ravdess'):
    for j in os.listdir('./audios/ravdess/' + i):
        filename = './audios/ravdess/' + i + '/' + j
        print('./audios/ravdess/' + i + '/' + j)
        smile = opensmile.Smile(
              feature_set=opensmile.FeatureSet.eGeMAPSv02,
              feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
        )

        feats = smile.process_file(filename,channel=0)
        feats = feats.reset_index()
        feat_aux.append(feats)
        
end = time.time()
print(f"Tiempo: { round(end - start, 2) } segundos")

./audios/ravdess/Actor_23/03-01-07-02-02-01-23.wav
./audios/ravdess/Actor_23/03-02-03-02-02-01-23.wav
./audios/ravdess/Actor_23/03-01-04-01-02-01-23.wav
./audios/ravdess/Actor_23/03-02-04-02-02-01-23.wav
./audios/ravdess/Actor_23/03-01-02-02-02-02-23.wav
./audios/ravdess/Actor_23/03-02-05-01-01-02-23.wav
./audios/ravdess/Actor_23/03-02-05-02-02-02-23.wav
./audios/ravdess/Actor_23/03-01-07-02-02-02-23.wav
./audios/ravdess/Actor_23/03-02-02-01-02-01-23.wav
./audios/ravdess/Actor_23/03-02-04-01-01-02-23.wav
./audios/ravdess/Actor_23/03-02-06-02-02-02-23.wav
./audios/ravdess/Actor_23/03-01-02-02-02-01-23.wav
./audios/ravdess/Actor_23/03-02-03-02-01-02-23.wav
./audios/ravdess/Actor_23/03-01-03-01-02-01-23.wav
./audios/ravdess/Actor_23/03-01-05-01-02-02-23.wav
./audios/ravdess/Actor_23/03-02-05-01-02-02-23.wav
./audios/ravdess/Actor_23/03-01-08-02-01-01-23.wav
./audios/ravdess/Actor_23/03-02-02-01-02-02-23.wav
./audios/ravdess/Actor_23/03-01-02-01-01-01-23.wav
./audios/ravdess/Actor_23/03-02

./audios/ravdess/Actor_01/03-02-04-01-02-01-01.wav
./audios/ravdess/Actor_01/03-01-02-02-01-01-01.wav
./audios/ravdess/Actor_01/03-01-06-02-01-01-01.wav
./audios/ravdess/Actor_01/03-01-03-01-01-02-01.wav
./audios/ravdess/Actor_01/03-02-03-01-01-01-01.wav
./audios/ravdess/Actor_01/03-02-02-01-02-02-01.wav
./audios/ravdess/Actor_01/03-01-08-02-01-01-01.wav
./audios/ravdess/Actor_01/03-01-01-01-01-02-01.wav
./audios/ravdess/Actor_01/03-02-01-01-02-01-01.wav
./audios/ravdess/Actor_01/03-02-04-01-01-02-01.wav
./audios/ravdess/Actor_01/03-02-03-01-01-02-01.wav
./audios/ravdess/Actor_01/03-01-06-02-02-02-01.wav
./audios/ravdess/Actor_01/03-01-07-02-02-02-01.wav
./audios/ravdess/Actor_01/03-01-02-02-01-02-01.wav
./audios/ravdess/Actor_01/03-02-02-02-02-01-01.wav
./audios/ravdess/Actor_01/03-01-05-01-02-02-01.wav
./audios/ravdess/Actor_01/03-02-02-02-02-02-01.wav
./audios/ravdess/Actor_01/03-02-04-01-02-02-01.wav
./audios/ravdess/Actor_01/03-01-07-01-01-01-01.wav
./audios/ravdess/Actor_01/03-02

./audios/ravdess/Actor_16/03-01-04-02-01-02-16.wav
./audios/ravdess/Actor_16/03-01-06-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-04-02-02-02-16.wav
./audios/ravdess/Actor_16/03-01-07-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-08-02-02-01-16.wav
./audios/ravdess/Actor_16/03-02-01-01-02-01-16.wav
./audios/ravdess/Actor_16/03-01-04-01-01-02-16.wav
./audios/ravdess/Actor_16/03-01-06-02-02-01-16.wav
./audios/ravdess/Actor_16/03-02-03-02-01-02-16.wav
./audios/ravdess/Actor_16/03-02-03-01-01-01-16.wav
./audios/ravdess/Actor_16/03-02-02-01-02-02-16.wav
./audios/ravdess/Actor_16/03-01-02-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-01-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-02-02-02-01-16.wav
./audios/ravdess/Actor_16/03-02-02-02-02-02-16.wav
./audios/ravdess/Actor_16/03-02-05-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-03-01-02-01-16.wav
./audios/ravdess/Actor_16/03-02-05-01-01-02-16.wav
./audios/ravdess/Actor_16/03-01-06-01-01-02-16.wav
./audios/ravdess/Actor_16/03-01

./audios/ravdess/Actor_10/03-01-02-02-02-01-10.wav
./audios/ravdess/Actor_10/03-01-08-02-02-02-10.wav
./audios/ravdess/Actor_10/03-02-04-02-02-02-10.wav
./audios/ravdess/Actor_10/03-02-04-01-01-02-10.wav
./audios/ravdess/Actor_10/03-02-03-02-02-01-10.wav
./audios/ravdess/Actor_10/03-01-02-02-02-02-10.wav
./audios/ravdess/Actor_10/03-01-02-02-01-01-10.wav
./audios/ravdess/Actor_10/03-01-04-02-01-01-10.wav
./audios/ravdess/Actor_10/03-01-05-02-02-02-10.wav
./audios/ravdess/Actor_10/03-02-06-01-01-01-10.wav
./audios/ravdess/Actor_10/03-02-06-01-02-01-10.wav
./audios/ravdess/Actor_10/03-02-04-01-02-01-10.wav
./audios/ravdess/Actor_10/03-01-07-01-01-01-10.wav
./audios/ravdess/Actor_10/03-02-01-01-02-01-10.wav
./audios/ravdess/Actor_10/03-02-03-02-01-02-10.wav
./audios/ravdess/Actor_10/03-02-02-01-02-01-10.wav
./audios/ravdess/Actor_10/03-02-05-01-01-02-10.wav
./audios/ravdess/Actor_10/03-01-06-02-01-02-10.wav
./audios/ravdess/Actor_10/03-02-04-02-02-01-10.wav
./audios/ravdess/Actor_10/03-01

./audios/ravdess/Actor_09/03-01-08-02-01-02-09.wav
./audios/ravdess/Actor_09/03-01-01-01-02-01-09.wav
./audios/ravdess/Actor_09/03-01-05-02-02-01-09.wav
./audios/ravdess/Actor_09/03-01-07-02-02-02-09.wav
./audios/ravdess/Actor_09/03-02-02-01-01-01-09.wav
./audios/ravdess/Actor_09/03-01-08-01-02-02-09.wav
./audios/ravdess/Actor_09/03-01-06-01-02-02-09.wav
./audios/ravdess/Actor_09/03-02-05-02-02-01-09.wav
./audios/ravdess/Actor_09/03-01-05-01-01-02-09.wav
./audios/ravdess/Actor_09/03-01-02-01-01-01-09.wav
./audios/ravdess/Actor_09/03-02-03-01-02-01-09.wav
./audios/ravdess/Actor_09/03-01-08-01-01-01-09.wav
./audios/ravdess/Actor_09/03-02-04-01-01-02-09.wav
./audios/ravdess/Actor_09/03-01-03-02-02-01-09.wav
./audios/ravdess/Actor_09/03-02-06-01-01-01-09.wav
./audios/ravdess/Actor_09/03-02-02-02-02-02-09.wav
./audios/ravdess/Actor_09/03-02-03-01-01-01-09.wav
./audios/ravdess/Actor_09/03-01-04-01-01-01-09.wav
./audios/ravdess/Actor_09/03-01-08-01-01-02-09.wav
./audios/ravdess/Actor_09/03-01

...

./audios/ravdess/Actor_06/03-01-05-02-01-02-06.wav
./audios/ravdess/Actor_06/03-01-07-01-02-01-06.wav
./audios/ravdess/Actor_06/03-02-05-02-02-02-06.wav
./audios/ravdess/Actor_06/03-01-05-02-02-01-06.wav
./audios/ravdess/Actor_06/03-01-06-01-01-01-06.wav
./audios/ravdess/Actor_06/03-02-03-01-01-02-06.wav
./audios/ravdess/Actor_06/03-01-01-01-02-02-06.wav
./audios/ravdess/Actor_06/03-02-02-01-02-01-06.wav
./audios/ravdess/Actor_06/03-01-02-01-02-02-06.wav
./audios/ravdess/Actor_06/03-02-03-01-02-01-06.wav
./audios/ravdess/Actor_06/03-01-03-01-01-01-06.wav
./audios/ravdess/Actor_06/03-02-02-02-01-02-06.wav
./audios/ravdess/Actor_06/03-02-04-01-02-02-06.wav
./audios/ravdess/Actor_06/03-01-04-02-02-01-06.wav
./audios/ravdess/Actor_06/03-02-01-01-01-01-06.wav
./audios/ravdess/Actor_06/03-01-08-01-02-01-06.wav
./audios/ravdess/Actor_06/03-02-06-02-01-02-06.wav
./audios/ravdess/Actor_06/03-01-03-02-01-02-06.wav
./audios/ravdess/Actor_06/03-02-05-01-02-01-06.wav
./audios/ravdess/Actor_06/03-01

./audios/ravdess/Actor_05/03-01-04-02-02-02-05.wav
./audios/ravdess/Actor_05/03-02-03-02-01-02-05.wav
./audios/ravdess/Actor_05/03-01-04-01-01-02-05.wav
./audios/ravdess/Actor_05/03-01-05-01-02-01-05.wav
./audios/ravdess/Actor_05/03-01-03-01-01-02-05.wav
./audios/ravdess/Actor_05/03-02-02-02-02-01-05.wav
./audios/ravdess/Actor_05/03-02-03-01-01-02-05.wav
./audios/ravdess/Actor_05/03-01-07-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-03-01-02-02-05.wav
./audios/ravdess/Actor_05/03-02-04-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-05-02-02-02-05.wav
./audios/ravdess/Actor_05/03-02-02-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-02-02-01-01-05.wav
./audios/ravdess/Actor_05/03-02-05-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-05-02-01-01-05.wav
./audios/ravdess/Actor_05/03-01-06-01-01-01-05.wav
./audios/ravdess/Actor_05/03-02-01-01-01-02-05.wav
./audios/ravdess/Actor_05/03-01-07-02-01-02-05.wav
./audios/ravdess/Actor_05/03-02-04-01-01-02-05.wav
./audios/ravdess/Actor_05/03-02

./audios/ravdess/Actor_07/03-02-03-02-02-02-07.wav
./audios/ravdess/Actor_07/03-02-02-01-02-02-07.wav
./audios/ravdess/Actor_07/03-01-05-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-07-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-05-02-01-01-07.wav
./audios/ravdess/Actor_07/03-01-03-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-07-01-01-01-07.wav
./audios/ravdess/Actor_07/03-02-04-01-02-02-07.wav
./audios/ravdess/Actor_07/03-01-04-01-01-01-07.wav
./audios/ravdess/Actor_07/03-02-04-02-02-02-07.wav
./audios/ravdess/Actor_07/03-01-04-02-01-02-07.wav
./audios/ravdess/Actor_07/03-01-02-02-02-01-07.wav
./audios/ravdess/Actor_07/03-02-03-01-02-02-07.wav
./audios/ravdess/Actor_07/03-01-08-01-02-02-07.wav
./audios/ravdess/Actor_07/03-02-03-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-08-02-02-01-07.wav
./audios/ravdess/Actor_07/03-01-06-02-01-02-07.wav
./audios/ravdess/Actor_07/03-02-05-01-02-01-07.wav
./audios/ravdess/Actor_07/03-02-01-01-01-01-07.wav
./audios/ravdess/Actor_07/03-02

./audios/ravdess/Actor_20/03-01-05-02-02-02-20.wav
./audios/ravdess/Actor_20/03-01-07-01-02-01-20.wav
./audios/ravdess/Actor_20/03-01-03-01-01-01-20.wav
./audios/ravdess/Actor_20/03-01-06-02-01-02-20.wav
./audios/ravdess/Actor_20/03-02-04-02-02-02-20.wav
./audios/ravdess/Actor_20/03-02-03-01-02-02-20.wav
./audios/ravdess/Actor_20/03-01-06-01-01-02-20.wav
./audios/ravdess/Actor_20/03-01-05-02-01-01-20.wav
./audios/ravdess/Actor_20/03-01-05-01-01-02-20.wav
./audios/ravdess/Actor_20/03-02-06-02-01-02-20.wav
./audios/ravdess/Actor_20/03-01-04-02-01-01-20.wav
./audios/ravdess/Actor_20/03-02-04-02-01-01-20.wav
./audios/ravdess/Actor_20/03-01-05-02-02-01-20.wav
./audios/ravdess/Actor_20/03-02-02-01-02-02-20.wav
./audios/ravdess/Actor_20/03-01-03-01-02-01-20.wav
./audios/ravdess/Actor_20/03-02-03-02-01-02-20.wav
./audios/ravdess/Actor_20/03-01-06-02-01-01-20.wav
./audios/ravdess/Actor_20/03-02-04-01-01-02-20.wav
./audios/ravdess/Actor_20/03-01-06-01-01-01-20.wav
./audios/ravdess/Actor_20/03-01

./audios/ravdess/Actor_22/03-01-05-02-02-02-22.wav
./audios/ravdess/Actor_22/03-01-08-01-02-01-22.wav
./audios/ravdess/Actor_22/03-02-03-02-01-02-22.wav
./audios/ravdess/Actor_22/03-02-02-01-02-02-22.wav
./audios/ravdess/Actor_22/03-01-03-01-01-02-22.wav
./audios/ravdess/Actor_22/03-01-04-02-01-01-22.wav
./audios/ravdess/Actor_22/03-01-04-02-02-01-22.wav
./audios/ravdess/Actor_22/03-02-03-01-01-02-22.wav
./audios/ravdess/Actor_22/03-02-05-02-02-02-22.wav
./audios/ravdess/Actor_22/03-01-06-02-01-02-22.wav
./audios/ravdess/Actor_22/03-01-06-01-02-02-22.wav
./audios/ravdess/Actor_22/03-01-07-02-02-02-22.wav
./audios/ravdess/Actor_22/03-01-02-01-01-02-22.wav
./audios/ravdess/Actor_22/03-01-01-01-02-02-22.wav
./audios/ravdess/Actor_22/03-02-03-02-01-01-22.wav
./audios/ravdess/Actor_22/03-02-03-02-02-01-22.wav
./audios/ravdess/Actor_22/03-02-06-02-02-01-22.wav
./audios/ravdess/Actor_22/03-01-07-01-01-02-22.wav
./audios/ravdess/Actor_22/03-01-08-01-02-02-22.wav
./audios/ravdess/Actor_22/03-01

In [9]:
df_feats = pd.concat(feat_aux)
df_feats = df_feats.rename(columns={'file': 'file_path'})

In [10]:
df_feats.shape

(992528, 28)

In [11]:
df_ravdess.head()

,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor
0,./audios/ravdess/Actor_23/03-01-07-02-02-01-23...,03,01,07,02,02,01,23
1,./audios/ravdess/Actor_23/03-02-03-02-02-01-23...,03,02,03,02,02,01,23
2,./audios/ravdess/Actor_23/03-01-04-01-02-01-23...,03,01,04,01,02,01,23
3,./audios/ravdess/Actor_23/03-02-04-02-02-01-23...,03,02,04,02,02,01,23
4,./audios/ravdess/Actor_23/03-01-02-02-02-02-23...,03,01,02,02,02,02,23


In [12]:
# df_join = pd.merge(df_ravdess, df_feats, on="file_path", how="left")

In [13]:
# df_join.head()

#### Atributos de alto nivel (Functionals)

Se calculan a partir de los atributos de bajo nivel, calculando estadísticas que resuman las secuencias. Los modelos que vimos no modelan secuencias!, por ende, es necesario disponer de atributos que consistan de vectores fijos.

In [14]:
start = time.time()

feat_aux = []
for i in os.listdir('./audios/ravdess'):
    for j in os.listdir('./audios/ravdess/' + i):
        filename = './audios/ravdess/' + i + '/' + j
        print('./audios/ravdess/' + i + '/' + j)
        smile = opensmile.Smile(
              feature_set=opensmile.FeatureSet.eGeMAPSv02,
              feature_level=opensmile.FeatureLevel.Functionals,
        )

        feats = smile.process_file(filename,channel=0)
        feats = feats.reset_index()
        feat_aux.append(feats)
        
end = time.time()
print(f"Tiempo: { round(end - start, 2) } segundos")

./audios/ravdess/Actor_23/03-01-07-02-02-01-23.wav
./audios/ravdess/Actor_23/03-02-03-02-02-01-23.wav
./audios/ravdess/Actor_23/03-01-04-01-02-01-23.wav
./audios/ravdess/Actor_23/03-02-04-02-02-01-23.wav
./audios/ravdess/Actor_23/03-01-02-02-02-02-23.wav
./audios/ravdess/Actor_23/03-02-05-01-01-02-23.wav
./audios/ravdess/Actor_23/03-02-05-02-02-02-23.wav
./audios/ravdess/Actor_23/03-01-07-02-02-02-23.wav
./audios/ravdess/Actor_23/03-02-02-01-02-01-23.wav
./audios/ravdess/Actor_23/03-02-04-01-01-02-23.wav
./audios/ravdess/Actor_23/03-02-06-02-02-02-23.wav
./audios/ravdess/Actor_23/03-01-02-02-02-01-23.wav
./audios/ravdess/Actor_23/03-02-03-02-01-02-23.wav
./audios/ravdess/Actor_23/03-01-03-01-02-01-23.wav
./audios/ravdess/Actor_23/03-01-05-01-02-02-23.wav
./audios/ravdess/Actor_23/03-02-05-01-02-02-23.wav
./audios/ravdess/Actor_23/03-01-08-02-01-01-23.wav
./audios/ravdess/Actor_23/03-02-02-01-02-02-23.wav
./audios/ravdess/Actor_23/03-01-02-01-01-01-23.wav
./audios/ravdess/Actor_23/03-02

./audios/ravdess/Actor_01/03-02-04-01-02-01-01.wav
./audios/ravdess/Actor_01/03-01-02-02-01-01-01.wav
./audios/ravdess/Actor_01/03-01-06-02-01-01-01.wav
./audios/ravdess/Actor_01/03-01-03-01-01-02-01.wav
./audios/ravdess/Actor_01/03-02-03-01-01-01-01.wav
./audios/ravdess/Actor_01/03-02-02-01-02-02-01.wav
./audios/ravdess/Actor_01/03-01-08-02-01-01-01.wav
./audios/ravdess/Actor_01/03-01-01-01-01-02-01.wav
./audios/ravdess/Actor_01/03-02-01-01-02-01-01.wav
./audios/ravdess/Actor_01/03-02-04-01-01-02-01.wav
./audios/ravdess/Actor_01/03-02-03-01-01-02-01.wav
./audios/ravdess/Actor_01/03-01-06-02-02-02-01.wav
./audios/ravdess/Actor_01/03-01-07-02-02-02-01.wav
./audios/ravdess/Actor_01/03-01-02-02-01-02-01.wav
./audios/ravdess/Actor_01/03-02-02-02-02-01-01.wav
./audios/ravdess/Actor_01/03-01-05-01-02-02-01.wav
./audios/ravdess/Actor_01/03-02-02-02-02-02-01.wav
./audios/ravdess/Actor_01/03-02-04-01-02-02-01.wav
./audios/ravdess/Actor_01/03-01-07-01-01-01-01.wav
./audios/ravdess/Actor_01/03-02

./audios/ravdess/Actor_16/03-01-04-02-01-02-16.wav
./audios/ravdess/Actor_16/03-01-06-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-04-02-02-02-16.wav
./audios/ravdess/Actor_16/03-01-07-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-08-02-02-01-16.wav
./audios/ravdess/Actor_16/03-02-01-01-02-01-16.wav
./audios/ravdess/Actor_16/03-01-04-01-01-02-16.wav
./audios/ravdess/Actor_16/03-01-06-02-02-01-16.wav
./audios/ravdess/Actor_16/03-02-03-02-01-02-16.wav
./audios/ravdess/Actor_16/03-02-03-01-01-01-16.wav
./audios/ravdess/Actor_16/03-02-02-01-02-02-16.wav
./audios/ravdess/Actor_16/03-01-02-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-01-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-02-02-02-01-16.wav
./audios/ravdess/Actor_16/03-02-02-02-02-02-16.wav
./audios/ravdess/Actor_16/03-02-05-01-01-01-16.wav
./audios/ravdess/Actor_16/03-01-03-01-02-01-16.wav
./audios/ravdess/Actor_16/03-02-05-01-01-02-16.wav
./audios/ravdess/Actor_16/03-01-06-01-01-02-16.wav
./audios/ravdess/Actor_16/03-01

./audios/ravdess/Actor_10/03-01-02-02-02-01-10.wav
./audios/ravdess/Actor_10/03-01-08-02-02-02-10.wav
./audios/ravdess/Actor_10/03-02-04-02-02-02-10.wav
./audios/ravdess/Actor_10/03-02-04-01-01-02-10.wav
./audios/ravdess/Actor_10/03-02-03-02-02-01-10.wav
./audios/ravdess/Actor_10/03-01-02-02-02-02-10.wav
./audios/ravdess/Actor_10/03-01-02-02-01-01-10.wav
./audios/ravdess/Actor_10/03-01-04-02-01-01-10.wav
./audios/ravdess/Actor_10/03-01-05-02-02-02-10.wav
./audios/ravdess/Actor_10/03-02-06-01-01-01-10.wav
./audios/ravdess/Actor_10/03-02-06-01-02-01-10.wav
./audios/ravdess/Actor_10/03-02-04-01-02-01-10.wav
./audios/ravdess/Actor_10/03-01-07-01-01-01-10.wav
./audios/ravdess/Actor_10/03-02-01-01-02-01-10.wav
./audios/ravdess/Actor_10/03-02-03-02-01-02-10.wav
./audios/ravdess/Actor_10/03-02-02-01-02-01-10.wav
./audios/ravdess/Actor_10/03-02-05-01-01-02-10.wav
./audios/ravdess/Actor_10/03-01-06-02-01-02-10.wav
./audios/ravdess/Actor_10/03-02-04-02-02-01-10.wav
./audios/ravdess/Actor_10/03-01

./audios/ravdess/Actor_09/03-01-08-02-01-02-09.wav
./audios/ravdess/Actor_09/03-01-01-01-02-01-09.wav
./audios/ravdess/Actor_09/03-01-05-02-02-01-09.wav
./audios/ravdess/Actor_09/03-01-07-02-02-02-09.wav
./audios/ravdess/Actor_09/03-02-02-01-01-01-09.wav
./audios/ravdess/Actor_09/03-01-08-01-02-02-09.wav
./audios/ravdess/Actor_09/03-01-06-01-02-02-09.wav
./audios/ravdess/Actor_09/03-02-05-02-02-01-09.wav
./audios/ravdess/Actor_09/03-01-05-01-01-02-09.wav
./audios/ravdess/Actor_09/03-01-02-01-01-01-09.wav
./audios/ravdess/Actor_09/03-02-03-01-02-01-09.wav
./audios/ravdess/Actor_09/03-01-08-01-01-01-09.wav
./audios/ravdess/Actor_09/03-02-04-01-01-02-09.wav
./audios/ravdess/Actor_09/03-01-03-02-02-01-09.wav
./audios/ravdess/Actor_09/03-02-06-01-01-01-09.wav
./audios/ravdess/Actor_09/03-02-02-02-02-02-09.wav
./audios/ravdess/Actor_09/03-02-03-01-01-01-09.wav
./audios/ravdess/Actor_09/03-01-04-01-01-01-09.wav
./audios/ravdess/Actor_09/03-01-08-01-01-02-09.wav
./audios/ravdess/Actor_09/03-01

...

./audios/ravdess/Actor_06/03-01-01-01-02-01-06.wav
./audios/ravdess/Actor_06/03-01-08-02-02-02-06.wav
./audios/ravdess/Actor_06/03-01-05-02-01-02-06.wav
./audios/ravdess/Actor_06/03-01-07-01-02-01-06.wav
./audios/ravdess/Actor_06/03-02-05-02-02-02-06.wav
./audios/ravdess/Actor_06/03-01-05-02-02-01-06.wav
./audios/ravdess/Actor_06/03-01-06-01-01-01-06.wav
./audios/ravdess/Actor_06/03-02-03-01-01-02-06.wav
./audios/ravdess/Actor_06/03-01-01-01-02-02-06.wav
./audios/ravdess/Actor_06/03-02-02-01-02-01-06.wav
./audios/ravdess/Actor_06/03-01-02-01-02-02-06.wav
./audios/ravdess/Actor_06/03-02-03-01-02-01-06.wav
./audios/ravdess/Actor_06/03-01-03-01-01-01-06.wav
./audios/ravdess/Actor_06/03-02-02-02-01-02-06.wav
./audios/ravdess/Actor_06/03-02-04-01-02-02-06.wav
./audios/ravdess/Actor_06/03-01-04-02-02-01-06.wav
./audios/ravdess/Actor_06/03-02-01-01-01-01-06.wav
./audios/ravdess/Actor_06/03-01-08-01-02-01-06.wav
./audios/ravdess/Actor_06/03-02-06-02-01-02-06.wav
./audios/ravdess/Actor_06/03-01

./audios/ravdess/Actor_05/03-02-04-01-02-01-05.wav
./audios/ravdess/Actor_05/03-02-01-01-01-01-05.wav
./audios/ravdess/Actor_05/03-01-04-02-02-02-05.wav
./audios/ravdess/Actor_05/03-02-03-02-01-02-05.wav
./audios/ravdess/Actor_05/03-01-04-01-01-02-05.wav
./audios/ravdess/Actor_05/03-01-05-01-02-01-05.wav
./audios/ravdess/Actor_05/03-01-03-01-01-02-05.wav
./audios/ravdess/Actor_05/03-02-02-02-02-01-05.wav
./audios/ravdess/Actor_05/03-02-03-01-01-02-05.wav
./audios/ravdess/Actor_05/03-01-07-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-03-01-02-02-05.wav
./audios/ravdess/Actor_05/03-02-04-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-05-02-02-02-05.wav
./audios/ravdess/Actor_05/03-02-02-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-02-02-01-01-05.wav
./audios/ravdess/Actor_05/03-02-05-02-02-02-05.wav
./audios/ravdess/Actor_05/03-01-05-02-01-01-05.wav
./audios/ravdess/Actor_05/03-01-06-01-01-01-05.wav
./audios/ravdess/Actor_05/03-02-01-01-01-02-05.wav
./audios/ravdess/Actor_05/03-01

./audios/ravdess/Actor_07/03-01-05-01-01-01-07.wav
./audios/ravdess/Actor_07/03-02-05-01-01-02-07.wav
./audios/ravdess/Actor_07/03-02-03-02-02-02-07.wav
./audios/ravdess/Actor_07/03-02-02-01-02-02-07.wav
./audios/ravdess/Actor_07/03-01-05-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-07-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-05-02-01-01-07.wav
./audios/ravdess/Actor_07/03-01-03-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-07-01-01-01-07.wav
./audios/ravdess/Actor_07/03-02-04-01-02-02-07.wav
./audios/ravdess/Actor_07/03-01-04-01-01-01-07.wav
./audios/ravdess/Actor_07/03-02-04-02-02-02-07.wav
./audios/ravdess/Actor_07/03-01-04-02-01-02-07.wav
./audios/ravdess/Actor_07/03-01-02-02-02-01-07.wav
./audios/ravdess/Actor_07/03-02-03-01-02-02-07.wav
./audios/ravdess/Actor_07/03-01-08-01-02-02-07.wav
./audios/ravdess/Actor_07/03-02-03-01-02-01-07.wav
./audios/ravdess/Actor_07/03-01-08-02-02-01-07.wav
./audios/ravdess/Actor_07/03-01-06-02-01-02-07.wav
./audios/ravdess/Actor_07/03-02

./audios/ravdess/Actor_20/03-02-03-01-01-01-20.wav
./audios/ravdess/Actor_20/03-02-06-02-01-01-20.wav
./audios/ravdess/Actor_20/03-01-05-02-02-02-20.wav
./audios/ravdess/Actor_20/03-01-07-01-02-01-20.wav
./audios/ravdess/Actor_20/03-01-03-01-01-01-20.wav
./audios/ravdess/Actor_20/03-01-06-02-01-02-20.wav
./audios/ravdess/Actor_20/03-02-04-02-02-02-20.wav
./audios/ravdess/Actor_20/03-02-03-01-02-02-20.wav
./audios/ravdess/Actor_20/03-01-06-01-01-02-20.wav
./audios/ravdess/Actor_20/03-01-05-02-01-01-20.wav
./audios/ravdess/Actor_20/03-01-05-01-01-02-20.wav
./audios/ravdess/Actor_20/03-02-06-02-01-02-20.wav
./audios/ravdess/Actor_20/03-01-04-02-01-01-20.wav
./audios/ravdess/Actor_20/03-02-04-02-01-01-20.wav
./audios/ravdess/Actor_20/03-01-05-02-02-01-20.wav
./audios/ravdess/Actor_20/03-02-02-01-02-02-20.wav
./audios/ravdess/Actor_20/03-01-03-01-02-01-20.wav
./audios/ravdess/Actor_20/03-02-03-02-01-02-20.wav
./audios/ravdess/Actor_20/03-01-06-02-01-01-20.wav
./audios/ravdess/Actor_20/03-02

./audios/ravdess/Actor_22/03-02-02-01-01-02-22.wav
./audios/ravdess/Actor_22/03-02-04-02-01-01-22.wav
./audios/ravdess/Actor_22/03-01-05-02-02-02-22.wav
./audios/ravdess/Actor_22/03-01-08-01-02-01-22.wav
./audios/ravdess/Actor_22/03-02-03-02-01-02-22.wav
./audios/ravdess/Actor_22/03-02-02-01-02-02-22.wav
./audios/ravdess/Actor_22/03-01-03-01-01-02-22.wav
./audios/ravdess/Actor_22/03-01-04-02-01-01-22.wav
./audios/ravdess/Actor_22/03-01-04-02-02-01-22.wav
./audios/ravdess/Actor_22/03-02-03-01-01-02-22.wav
./audios/ravdess/Actor_22/03-02-05-02-02-02-22.wav
./audios/ravdess/Actor_22/03-01-06-02-01-02-22.wav
./audios/ravdess/Actor_22/03-01-06-01-02-02-22.wav
./audios/ravdess/Actor_22/03-01-07-02-02-02-22.wav
./audios/ravdess/Actor_22/03-01-02-01-01-02-22.wav
./audios/ravdess/Actor_22/03-01-01-01-02-02-22.wav
./audios/ravdess/Actor_22/03-02-03-02-01-01-22.wav
./audios/ravdess/Actor_22/03-02-03-02-02-01-22.wav
./audios/ravdess/Actor_22/03-02-06-02-02-01-22.wav
./audios/ravdess/Actor_22/03-01

In [15]:
df_feats = pd.concat(feat_aux)
df_feats = df_feats.rename(columns={'file': 'file_path'})

In [16]:
df_join = pd.merge(df_ravdess, df_feats, on="file_path", how="left")

In [17]:
df_join.head()

,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,start,end,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,./audios/ravdess/Actor_23/03-01-07-02-02-01-23...,03,01,07,02,02,01,23,0 days,0 days 00:00:03.737062500,...,0.075190,0.014015,0.022255,2.150538,1.089918,0.2825,0.126763,0.4900,0.397291,-36.658245
1,./audios/ravdess/Actor_23/03-02-03-02-02-01-23...,03,02,03,02,02,01,23,0 days,0 days 00:00:04.704687500,...,0.020998,0.009869,0.018871,1.492537,0.646552,0.8200,0.277248,0.5275,0.438770,-29.391884
2,./audios/ravdess/Actor_23/03-01-04-01-02-01-23...,03,01,04,01,02,01,23,0 days,0 days 00:00:03.570229167,...,0.047365,0.011048,0.011381,2.247191,0.569801,0.5900,0.160000,0.7600,0.296985,-43.332924
3,./audios/ravdess/Actor_23/03-02-04-02-02-01-23...,03,02,04,02,02,01,23,0 days,0 days 00:00:05.038354167,...,0.041743,0.012340,0.007390,1.394422,0.603622,0.9300,0.885701,0.5275,0.420260,-37.549572
4,./audios/ravdess/Actor_23/03-01-02-02-02-02-23...,03,01,02,02,02,02,23,0 days,0 days 00:00:04.571229167,...,0.053580,0.011590,0.005245,1.535088,0.886918,0.3875,0.206079,0.5740,0.424999,-43.261425


In [19]:
df_join.shape

(2452, 98)

In [20]:
df_join.to_csv('01.df.procesado.csv')

### Atributos de alto nivel (Functionals) - ComParE_2016

In [26]:
start = time.time()
import sox 

feat_aux = []
for i in os.listdir("./audios/ravdess"):
    for j in os.listdir("./audios/ravdess/" + i):
        filename = "./audios/ravdess/" + i + "/" + j
        print("./audios/ravdess/" + i + "/" + j)
        ch = sox.file_info.channels(filename)
        print(f"channel: {ch}")
        smile = opensmile.Smile(
            feature_set=opensmile.FeatureSet.ComParE_2016,
            feature_level=opensmile.FeatureLevel.Functionals,
            num_channels=ch,
        )

        feats = smile.process_file(
            filename
        )  # feats = smile.process_file(filename,channel=0)
        feats = feats.reset_index()
        feat_aux.append(feats)

end = time.time()
print(f"Tiempo: { round(end - start, 2) } segundos")

./audios/ravdess/Actor_23/03-01-07-02-02-01-23.wav
1
./audios/ravdess/Actor_23/03-02-03-02-02-01-23.wav
1
./audios/ravdess/Actor_23/03-01-04-01-02-01-23.wav
1
./audios/ravdess/Actor_23/03-02-04-02-02-01-23.wav
1
./audios/ravdess/Actor_23/03-01-02-02-02-02-23.wav
1
./audios/ravdess/Actor_23/03-02-05-01-01-02-23.wav
1
./audios/ravdess/Actor_23/03-02-05-02-02-02-23.wav
1
./audios/ravdess/Actor_23/03-01-07-02-02-02-23.wav
1
./audios/ravdess/Actor_23/03-02-02-01-02-01-23.wav
1
./audios/ravdess/Actor_23/03-02-04-01-01-02-23.wav
1
./audios/ravdess/Actor_23/03-02-06-02-02-02-23.wav
1
./audios/ravdess/Actor_23/03-01-02-02-02-01-23.wav
1
./audios/ravdess/Actor_23/03-02-03-02-01-02-23.wav
1
./audios/ravdess/Actor_23/03-01-03-01-02-01-23.wav
1
./audios/ravdess/Actor_23/03-01-05-01-02-02-23.wav
1
./audios/ravdess/Actor_23/03-02-05-01-02-02-23.wav
1
./audios/ravdess/Actor_23/03-01-08-02-01-01-23.wav
1
./audios/ravdess/Actor_23/03-02-02-01-02-02-23.wav
1
./audios/ravdess/Actor_23/03-01-02-01-01-01-23

1
./audios/ravdess/Actor_01/03-01-06-01-01-01-01.wav
1
./audios/ravdess/Actor_01/03-02-06-02-02-01-01.wav
1
./audios/ravdess/Actor_01/03-02-03-02-01-01-01.wav
1
./audios/ravdess/Actor_01/03-01-01-01-02-02-01.wav
1
./audios/ravdess/Actor_01/03-01-07-01-01-02-01.wav
1
./audios/ravdess/Actor_01/03-02-06-02-02-02-01.wav
1
./audios/ravdess/Actor_01/03-02-04-01-02-01-01.wav
1
./audios/ravdess/Actor_01/03-01-02-02-01-01-01.wav
1
./audios/ravdess/Actor_01/03-01-06-02-01-01-01.wav
1
./audios/ravdess/Actor_01/03-01-03-01-01-02-01.wav
1
./audios/ravdess/Actor_01/03-02-03-01-01-01-01.wav
1
./audios/ravdess/Actor_01/03-02-02-01-02-02-01.wav
1
./audios/ravdess/Actor_01/03-01-08-02-01-01-01.wav
1
./audios/ravdess/Actor_01/03-01-01-01-01-02-01.wav
1
./audios/ravdess/Actor_01/03-02-01-01-02-01-01.wav
1
./audios/ravdess/Actor_01/03-02-04-01-01-02-01.wav
1
./audios/ravdess/Actor_01/03-02-03-01-01-02-01.wav
1
./audios/ravdess/Actor_01/03-01-06-02-02-02-01.wav
1
./audios/ravdess/Actor_01/03-01-07-02-02-02-

1
./audios/ravdess/Actor_21/03-02-03-02-02-01-21.wav
1
./audios/ravdess/Actor_21/03-01-05-01-01-01-21.wav
1
./audios/ravdess/Actor_16/03-02-03-01-01-02-16.wav
1
./audios/ravdess/Actor_16/03-01-05-02-01-01-16.wav
1
./audios/ravdess/Actor_16/03-01-08-02-01-01-16.wav
1
./audios/ravdess/Actor_16/03-02-01-01-01-01-16.wav
1
./audios/ravdess/Actor_16/03-01-08-02-01-02-16.wav
1
./audios/ravdess/Actor_16/03-02-03-01-02-02-16.wav
1
./audios/ravdess/Actor_16/03-01-08-01-02-01-16.wav
1
./audios/ravdess/Actor_16/03-01-02-01-02-01-16.wav
1
./audios/ravdess/Actor_16/03-02-04-02-02-02-16.wav
1
./audios/ravdess/Actor_16/03-02-05-02-01-01-16.wav
1
./audios/ravdess/Actor_16/03-01-04-02-01-02-16.wav
1
./audios/ravdess/Actor_16/03-01-06-01-01-01-16.wav
1
./audios/ravdess/Actor_16/03-01-04-02-02-02-16.wav
1
./audios/ravdess/Actor_16/03-01-07-01-01-01-16.wav
1
./audios/ravdess/Actor_16/03-01-08-02-02-01-16.wav
1
./audios/ravdess/Actor_16/03-02-01-01-02-01-16.wav
1
./audios/ravdess/Actor_16/03-01-04-01-01-02-

./audios/ravdess/Actor_10/03-01-03-02-02-02-10.wav
1
./audios/ravdess/Actor_10/03-01-02-01-02-01-10.wav
1
./audios/ravdess/Actor_10/03-01-03-02-01-02-10.wav
1
./audios/ravdess/Actor_10/03-01-03-02-02-01-10.wav
1
./audios/ravdess/Actor_10/03-01-06-01-01-01-10.wav
1
./audios/ravdess/Actor_10/03-01-03-02-01-01-10.wav
1
./audios/ravdess/Actor_10/03-02-02-02-01-02-10.wav
1
./audios/ravdess/Actor_10/03-01-07-01-02-02-10.wav
1
./audios/ravdess/Actor_10/03-02-06-02-02-01-10.wav
1
./audios/ravdess/Actor_10/03-01-01-01-01-01-10.wav
1
./audios/ravdess/Actor_10/03-02-04-01-01-01-10.wav
1
./audios/ravdess/Actor_10/03-01-07-01-02-01-10.wav
1
./audios/ravdess/Actor_10/03-01-07-02-01-02-10.wav
1
./audios/ravdess/Actor_10/03-01-05-02-01-01-10.wav
1
./audios/ravdess/Actor_10/03-01-08-01-01-02-10.wav
1
./audios/ravdess/Actor_10/03-02-02-02-02-02-10.wav
1
./audios/ravdess/Actor_10/03-02-03-01-01-02-10.wav
1
./audios/ravdess/Actor_10/03-02-03-01-02-02-10.wav
1
./audios/ravdess/Actor_10/03-01-02-02-02-01-10

1
./audios/ravdess/Actor_12/03-01-01-01-01-02-12.wav
1
./audios/ravdess/Actor_12/03-01-03-01-01-02-12.wav
1
./audios/ravdess/Actor_12/03-01-07-02-01-01-12.wav
1
./audios/ravdess/Actor_12/03-01-03-01-01-01-12.wav
1
./audios/ravdess/Actor_09/03-02-03-01-01-02-09.wav
1
./audios/ravdess/Actor_09/03-01-06-02-01-02-09.wav
1
./audios/ravdess/Actor_09/03-01-05-01-01-01-09.wav
1
./audios/ravdess/Actor_09/03-01-03-02-02-02-09.wav
1
./audios/ravdess/Actor_09/03-01-05-02-01-02-09.wav
1
./audios/ravdess/Actor_09/03-02-06-01-01-02-09.wav
1
./audios/ravdess/Actor_09/03-02-01-01-01-01-09.wav
1
./audios/ravdess/Actor_09/03-01-02-01-02-02-09.wav
1
./audios/ravdess/Actor_09/03-02-05-01-02-02-09.wav
1
./audios/ravdess/Actor_09/03-02-05-01-02-01-09.wav
1
./audios/ravdess/Actor_09/03-01-05-01-02-02-09.wav
1
./audios/ravdess/Actor_09/03-01-04-02-02-02-09.wav
1
./audios/ravdess/Actor_09/03-01-01-01-02-02-09.wav
1
./audios/ravdess/Actor_09/03-02-03-02-01-02-09.wav
1
./audios/ravdess/Actor_09/03-01-04-01-01-02-

...

./audios/ravdess/Actor_17/03-02-01-01-01-01-17.wav
1
./audios/ravdess/Actor_17/03-02-03-02-02-01-17.wav
1
./audios/ravdess/Actor_17/03-01-07-01-02-01-17.wav
1
./audios/ravdess/Actor_17/03-01-08-01-01-02-17.wav
1
./audios/ravdess/Actor_17/03-01-05-02-01-01-17.wav
1
./audios/ravdess/Actor_17/03-02-04-01-02-01-17.wav
1
./audios/ravdess/Actor_17/03-02-02-02-02-01-17.wav
1
./audios/ravdess/Actor_17/03-01-06-02-01-02-17.wav
1
./audios/ravdess/Actor_17/03-01-05-02-02-01-17.wav
1
./audios/ravdess/Actor_17/03-01-08-01-02-02-17.wav
1
./audios/ravdess/Actor_17/03-01-04-02-01-02-17.wav
1
./audios/ravdess/Actor_17/03-01-03-01-01-02-17.wav
1
./audios/ravdess/Actor_17/03-01-02-02-02-02-17.wav
1
./audios/ravdess/Actor_17/03-01-03-02-01-01-17.wav
1
./audios/ravdess/Actor_17/03-01-01-01-01-01-17.wav
1
./audios/ravdess/Actor_17/03-01-08-02-02-02-17.wav
1
./audios/ravdess/Actor_17/03-01-07-02-01-02-17.wav
1
./audios/ravdess/Actor_17/03-02-01-01-01-02-17.wav
1
./audios/ravdess/Actor_17/03-01-03-02-01-02-17

./audios/ravdess/Actor_06/03-01-04-02-01-02-06.wav
1
./audios/ravdess/Actor_06/03-01-06-01-01-02-06.wav
1
./audios/ravdess/Actor_06/03-02-06-02-02-01-06.wav
1
./audios/ravdess/Actor_06/03-01-06-02-02-02-06.wav
1
./audios/ravdess/Actor_06/03-02-04-02-01-02-06.wav
1
./audios/ravdess/Actor_06/03-01-02-02-01-02-06.wav
1
./audios/ravdess/Actor_06/03-01-08-02-01-02-06.wav
1
./audios/ravdess/Actor_06/03-01-08-01-01-02-06.wav
1
./audios/ravdess/Actor_06/03-01-04-01-02-02-06.wav
1
./audios/ravdess/Actor_06/03-02-03-01-02-02-06.wav
1
./audios/ravdess/Actor_06/03-02-01-01-02-01-06.wav
1
./audios/ravdess/Actor_06/03-01-04-01-02-01-06.wav
1
./audios/ravdess/Actor_05/03-01-04-01-02-01-05.wav
1
./audios/ravdess/Actor_05/03-01-08-02-02-01-05.wav
1
./audios/ravdess/Actor_05/03-02-03-02-01-01-05.wav
1
./audios/ravdess/Actor_05/03-02-05-01-01-01-05.wav
1
./audios/ravdess/Actor_05/03-01-02-02-02-02-05.wav
1
./audios/ravdess/Actor_05/03-01-04-02-02-01-05.wav
1
./audios/ravdess/Actor_05/03-01-08-01-02-01-05

./audios/ravdess/Actor_18/03-01-04-02-02-02-18.wav
1
./audios/ravdess/Actor_18/03-01-05-01-02-02-18.wav
1
./audios/ravdess/Actor_18/03-01-08-01-01-01-18.wav
1
./audios/ravdess/Actor_18/03-01-03-02-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-01-01-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-05-01-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-03-01-02-02-18.wav
1
./audios/ravdess/Actor_18/03-01-05-02-01-01-18.wav
1
./audios/ravdess/Actor_18/03-01-07-01-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-06-02-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-05-02-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-04-01-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-06-02-02-02-18.wav
1
./audios/ravdess/Actor_18/03-01-08-02-02-02-18.wav
1
./audios/ravdess/Actor_18/03-01-07-01-02-01-18.wav
1
./audios/ravdess/Actor_18/03-01-02-02-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-02-01-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-08-01-01-02-18.wav
1
./audios/ravdess/Actor_18/03-01-03-02-02-02-18

1
./audios/ravdess/Actor_24/03-01-01-01-02-01-24.wav
1
./audios/ravdess/Actor_24/03-01-05-01-02-02-24.wav
1
./audios/ravdess/Actor_24/03-02-05-01-01-01-24.wav
1
./audios/ravdess/Actor_24/03-01-04-01-02-01-24.wav
1
./audios/ravdess/Actor_24/03-02-03-01-01-02-24.wav
1
./audios/ravdess/Actor_24/03-02-02-01-02-01-24.wav
1
./audios/ravdess/Actor_24/03-01-02-02-02-02-24.wav
1
./audios/ravdess/Actor_24/03-02-06-01-01-02-24.wav
1
./audios/ravdess/Actor_24/03-02-06-02-02-02-24.wav
1
./audios/ravdess/Actor_24/03-01-07-01-01-01-24.wav
1
./audios/ravdess/Actor_24/03-01-07-02-02-02-24.wav
1
./audios/ravdess/Actor_24/03-01-01-01-02-02-24.wav
1
./audios/ravdess/Actor_24/03-01-08-02-02-02-24.wav
1
./audios/ravdess/Actor_24/03-01-06-02-02-02-24.wav
1
./audios/ravdess/Actor_24/03-01-03-01-02-02-24.wav
1
./audios/ravdess/Actor_24/03-01-02-01-01-01-24.wav
1
./audios/ravdess/Actor_24/03-01-03-01-01-01-24.wav
1
./audios/ravdess/Actor_24/03-02-04-01-01-02-24.wav
1
./audios/ravdess/Actor_24/03-02-01-01-02-01-

./audios/ravdess/Actor_20/03-01-04-01-01-01-20.wav
1
./audios/ravdess/Actor_20/03-02-03-01-01-02-20.wav
1
./audios/ravdess/Actor_20/03-01-04-02-01-02-20.wav
1
./audios/ravdess/Actor_20/03-02-02-01-02-01-20.wav
1
./audios/ravdess/Actor_20/03-02-06-01-02-02-20.wav
1
./audios/ravdess/Actor_20/03-01-02-02-02-02-20.wav
1
./audios/ravdess/Actor_20/03-02-05-01-01-02-20.wav
1
./audios/ravdess/Actor_20/03-02-02-02-01-02-20.wav
1
./audios/ravdess/Actor_20/03-02-06-01-02-01-20.wav
1
./audios/ravdess/Actor_20/03-01-08-02-01-01-20.wav
1
./audios/ravdess/Actor_20/03-01-05-01-01-01-20.wav
1
./audios/ravdess/Actor_20/03-02-01-01-01-02-20.wav
1
./audios/ravdess/Actor_20/03-02-01-01-01-01-20.wav
1
./audios/ravdess/Actor_20/03-02-06-02-02-01-20.wav
1
./audios/ravdess/Actor_20/03-01-03-02-01-02-20.wav
1
./audios/ravdess/Actor_20/03-01-07-02-02-01-20.wav
1
./audios/ravdess/Actor_20/03-01-01-01-01-02-20.wav
1
./audios/ravdess/Actor_20/03-02-02-01-01-02-20.wav
1
./audios/ravdess/Actor_20/03-02-04-02-02-01-20

In [27]:
df_feats_ComParE_2016 = pd.concat(feat_aux)
df_feats_ComParE_2016 = df_feats_ComParE_2016.rename(columns={'file': 'file_path'})
df_feats_ComParE_2016 = pd.DataFrame(df_feats_ComParE_2016)
df_feats_ComParE_2016.shape
#df_feats_ComParE_2016.head()

(2452, 19122)

In [32]:
df_feats_ComParE_2016.head()

,file_path,start,end,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,...,mfcc_sma_de[14]_peakRangeAbs-1,mfcc_sma_de[14]_peakRangeRel-1,mfcc_sma_de[14]_peakMeanAbs-1,mfcc_sma_de[14]_peakMeanMeanDist-1,mfcc_sma_de[14]_peakMeanRel-1,mfcc_sma_de[14]_minRangeRel-1,mfcc_sma_de[14]_meanRisingSlope-1,mfcc_sma_de[14]_stddevRisingSlope-1,mfcc_sma_de[14]_meanFallingSlope-1,mfcc_sma_de[14]_stddevFallingSlope-1
0,./audios/ravdess/Actor_23/03-01-07-02-02-01-23...,0 days,0 days 00:00:03.737062500,1.494047,0.325137,0.024590,0.010098,0.030828,0.567017,0.020730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,./audios/ravdess/Actor_23/03-02-03-02-02-01-23...,0 days,0 days 00:00:04.704687500,2.579453,0.360691,0.995680,0.014117,0.560467,1.300311,0.546350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,./audios/ravdess/Actor_23/03-01-04-01-02-01-23...,0 days,0 days 00:00:03.570229167,0.930765,0.320000,0.925714,0.006846,0.018385,0.313856,0.011539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,./audios/ravdess/Actor_23/03-02-04-02-02-01-23...,0 days,0 days 00:00:05.038354167,1.285567,0.245968,0.104839,0.013745,0.161668,0.435290,0.147923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,./audios/ravdess/Actor_23/03-01-02-02-02-02-23...,0 days,0 days 00:00:04.571229167,1.070984,0.264444,0.000000,0.008435,0.022376,0.274260,0.013941,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_join_compare = pd.merge(df_ravdess, df_feats_ComParE_2016, on="file_path", how="left")

In [29]:
df_join_compare.head()

,file_path,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,start,end,...,mfcc_sma_de[14]_peakRangeAbs-1,mfcc_sma_de[14]_peakRangeRel-1,mfcc_sma_de[14]_peakMeanAbs-1,mfcc_sma_de[14]_peakMeanMeanDist-1,mfcc_sma_de[14]_peakMeanRel-1,mfcc_sma_de[14]_minRangeRel-1,mfcc_sma_de[14]_meanRisingSlope-1,mfcc_sma_de[14]_stddevRisingSlope-1,mfcc_sma_de[14]_meanFallingSlope-1,mfcc_sma_de[14]_stddevFallingSlope-1
0,./audios/ravdess/Actor_23/03-01-07-02-02-01-23...,03,01,07,02,02,01,23,0 days,0 days 00:00:03.737062500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,./audios/ravdess/Actor_23/03-02-03-02-02-01-23...,03,02,03,02,02,01,23,0 days,0 days 00:00:04.704687500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,./audios/ravdess/Actor_23/03-01-04-01-02-01-23...,03,01,04,01,02,01,23,0 days,0 days 00:00:03.570229167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,./audios/ravdess/Actor_23/03-02-04-02-02-01-23...,03,02,04,02,02,01,23,0 days,0 days 00:00:05.038354167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,./audios/ravdess/Actor_23/03-01-02-02-02-02-23...,03,01,02,02,02,02,23,0 days,0 days 00:00:04.571229167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_join_compare.shape

(2452, 19129)

In [31]:
df_join_compare.to_csv('01.df.procesado_ComParE_2016.csv')